In [26]:
from pathlib import Path

import pandas as pd
import numpy as np
import ray
import pyarrow as pa
import pyarrow.csv as pacsv

In [2]:
ray.init()

2023-09-27 14:46:18,300	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.18
Ray version:,2.7.0
Dashboard:,http://127.0.0.1:8265


In [4]:
filepaths = [
    str(p) for p in 
    Path("./day_5_data").rglob("amazon_us_reviews-train-00004-of-00005.parquet") 
]
df = ray.data.read_parquet(*filepaths)

(_get_reader pid=13421) /Users/vince/miniconda3/envs/cloner39/lib/python3.9/site-packages/ray/data/datasource/parquet_datasource.py:259: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_reader pid=13421)   pq_ds.pieces, **prefetch_remote_args
(_get_reader pid=13421) /Users/vince/miniconda3/envs/cloner39/lib/python3.9/site-packages/ray/data/datasource/parquet_datasource.py:269: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_reader pid=13421)   self._pq_pieces = [_SerializedPiece(p) for p in pq_ds.pieces]
(_get_reader pid=13421) /Users/vince/miniconda3/envs/cloner39/lib/python3.9/site-packages/ray/data/datasource/parquet_datasource.py:270: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a futu

(_get_reader pid=13421) [dataset]: Run `pip install tqdm` to enable progress reporting.


2023-09-27 14:48:11,656	INFO read_api.py:406 -- To satisfy the requested parallelism of 200, each read task output is split into 200 smaller blocks.


In [5]:
df.schema(False)

Column             Type
------             ----
marketplace        string
customer_id        string
review_id          string
product_id         string
product_parent     string
product_title      string
product_category   string
star_rating        int32
helpful_votes      int32
total_votes        int32
vine               int64
verified_purchase  int64
review_headline    string
review_body        string
review_date        string

In [6]:
df.show(2)

2023-09-27 14:48:46,534	INFO dataset.py:2380 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2023-09-27 14:48:46,537	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet->SplitBlocks(200)] -> LimitOperator[limit=2]
2023-09-27 14:48:46,537	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-09-27 14:48:46,538	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


[dataset]: Run `pip install tqdm` to enable progress reporting.
{'marketplace': 'US', 'customer_id': '31771169', 'review_id': 'R3OXY6IBXBIDNS', 'product_id': 'B001LRMV9O', 'product_parent': '895378446', 'product_title': "Carhartt Men's Sherpa Lined Sandstone Rugged Vest V26", 'product_category': 'Apparel', 'star_rating': 5, 'helpful_votes': 17, 'total_votes': 18, 'vine': 0, 'verified_purchase': 1, 'review_headline': 'Carhartt Vest', 'review_body': 'The vest is everything I was expecting from Carhartt. It is perfect for my activities as it keeps my core warm while giving me freedom of movement. I would caution that the sizes seem to run small. I normally wear a large, but ordered a X-large and it is a good fit. Would definitely recommend this product.', 'review_date': '2012-01-06'}
{'marketplace': 'US', 'customer_id': '9319708', 'review_id': 'R3M975LTQKIN9B', 'product_id': 'B001LRMPSG', 'product_parent': '138772958', 'product_title': "Carhartt Men's Loose Fit Five Pocket Canvas Carpente

In [45]:
# Each group is like {key: ['a','a'], value: [1,2]}
# All groups are like [ {key: ['a','a'], value: [1,2]}, ... ]
prod_allReviews_df = (
    df
    .select_columns(['product_id','review_id'])
    .groupby('product_id')
    .map_groups(
        lambda g: {'gr':[g['product_id'][0]], 'res':[g['review_id']]})
)

# All reviews reference DF
review_ref_df = (
    df
    .select_columns(['review_id'])
)

2023-09-28 12:09:14,432	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet->SplitBlocks(200)] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> LimitOperator[limit=1]
2023-09-28 12:09:14,432	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-09-28 12:09:14,433	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


In [37]:
take = prod_allReviews_df.take(1)

2023-09-27 15:28:00,321	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet->SplitBlocks(200)] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> AllToAllOperator[Sort] -> TaskPoolMapOperator[MapBatches(group_fn)] -> LimitOperator[limit=1]
2023-09-27 15:28:00,323	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=True, actor_locality_enabled=True, verbose_progress=False)
2023-09-27 15:28:00,324	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


In [48]:
take[0]['res']

0     R2THEEBLJRYPRH
1     R2ORDL3TCNYNHC
2      R72JS4DLXX6HC
3     R13L3OFZP8W4KA
4     R3DKTHZHWAE2DE
5     R3RLUPTMQTO663
6     R3638LBLGKMM2S
7      RP5AEIYZM5BUW
8     R37J4ZRGQ6BE30
9     R3T3ZDI9DP37QT
10     RTQWFOBWOJCOY
11    R3JHVB9WVQ8KHR
12    R1B6NFTKQHN2K1
13    R3PB8K42BMNI40
14    R2QD0XX7MSGT8B
15    R2PK1N9NG1JM4C
16    R2JX7J8YOWQJV9
Name: review_id, dtype: object

In [49]:
item_reviews = ray.data.from_items(take[0]['res'])

In [ ]:
def sample_irr_reviews(row):
    this_reviews_df = ray.data.from_items(row['res'])